# answers 01: _the adventure of the dark genomes_

## our one lonely import

As discussed in lecture, we're told to use almost entirely basic python (i.e. not even any of those 200+ modules in the Python standard library), with one exception: we can use NumPy, not because we need NumPy arrays, but because we need random number generation. So we import NumPy and initialize a default random number generator (RNG). I'll use this RNG when I get to generating random DNA negative control sequence.


In [8]:
import numpy as np

rng = np.random.default_rng()          # no argument = let the RNG seed itself randomly.

## 1. write a function to calculate the long ORF coverage statistic

The core of the pset is to be able to take a DNA sequence as input, and count the length of all long open reading frames (ORFs) in all six frames (including the reverse complement strand), where "long" means satisfying a minimum ORF length threshold we'll provide. We divide the total length of all these ORFs (in nucleotides) by the length of the genome to obtain a fractional coverage of the genome. The coverage statistic ranged from 0 to 6: it can be greater than 1 because there can be overlapping long ORFs in different frames/strands.

We know we're going to need to do this for different genetic codes that use different sets of stop codons, so we'll take the list of stop codons as an argument to our function too. We don't need a complete genetic code, because we're only counting ORF length, not translating; we only care about which codons are stops, and all the rest are sense codons.

We're also counting an ORF as literally an "open reading frame": a series of sense codons uninterrupted by a stop codon. We're not requiring that an ORF start with a canonical ATG start codon. (Pretty sure the pset would still work if we did, but we're not.)


In [2]:
def long_orf_coverage(seq, stopcodons, min_orflen):
    """Returns fraction of <seq> covered by long ORFs.
    
    Counts length of all ORFs in <seq> that are at least <min_orflen> long,
    in all six frames. Stop codons used for the six-frame translation are 
    defined by a list of uppercase strings in <stopcodons>: for example, 
    'TAG', 'TAA', 'TGA' ]. ORFs can start with any sense codon (not just Met).

    Returns the fractional coverage: the ratio of this total ORF length (in nucleotides) 
    over the length of the genome. This statistic ranges from 0 to 6, because
    we independently count overlapping ORFs in different frames/strands toward the
    total ORF length.

    <seq> is a DNA string all in upper case, [ACGT]+.
    """
    L      = len(seq)          # L is in nucleotides
    aatot  = 0
    for strand in range(0,2):  # First time through this loop is fwd strand; second time through is reverse complement.
        if strand == 1:
            seq = seq.translate(str.maketrans('ACGT','TGCA'))[::-1]    # reverse complement <seq>. tricksiest bit. see explanation below.

        for frame in range(0,3):   # looping over frames 0,1,2...
            orflen = 0
            for i in range(frame, L-2, 3):     # loop over all triplets, starting at <frame>, ending at L-3,L-2,L-1; seq is 0..L-1.
                if seq[i:i+3] in stopcodons:   # seq_i..i+2 triplet. get used to array/string indexing being on half-open intervals: [a,b). 
                    if orflen >= min_orflen: aatot += orflen
                    orflen = 0
                else:
                    orflen += 1
            if orflen >= min_orflen: aatot += orflen    # also count an ORF if it runs off the end without reaching a STOP
    coverage = aatot * 3 / L   # aatot is in amino acids, so multiply by three to get a nt/nt fraction
    return coverage


## 2. Do negative synthetic DNA controls and explain the SEA-PHAGES

Many of the 4924 SEA-PHAGE genomes showed surprisingly high long ORF coverages of higher than 1, meaning overlapping long ORFs in different frames. Wiggins' nice strip plot made it clear that long ORF coverage is correlated with GC% content of the genome. High GC% genomes have higher long ORF coverage. They're phage genomes, so we expected them to be compact and full of long ORFs, sure, but not that high.

Is there some weird biology doing on, or is this simply true for any old synthetic random DNA sequence we make, with varying GC% composition? We can make a function to generate a negative control synthetic DNA sequence. We learned just enough about NumPy random sampling to be able to generate a random DNA sequence of any GC% composition we want. 

Technically what we're making here is called an **i.i.d. random sequence**, where "i.i.d." means independent and identically distributed: each element (nucleotide) in the sequence has been sampled independently, from the same (multinomial) distribution. 

In [3]:
def make_random_dna(rng, L, frac_gc):
    """Returns an i.i.d. random DNA sequence as a string, with a given GC composition

    <rng> is a NumPy RNG
    <frac_gc> is the fractional G+C composition, in range [0,1].
    """
    composition = [ (1.0-frac_gc)/2, frac_gc/2, frac_gc/2, (1.0-frac_gc)/2 ]
    seq = ''.join(rng.choice(list('ACGT'), L, p=composition))
    return(seq)
    

Let's test that that works.

In [4]:
L   = 100000
seq = make_random_dna(rng, L, 0.7)                                # 70% GC composition
coverage = long_orf_coverage(seq, [ 'TAG', 'TAA', 'TGA' ], 200)   # require long ORFs >= 200
print(coverage)

0.64386


Yes, it's working. 

I can get a feel for what's going on by editing and rerunning that cell with different parameters, such as the GC%, the min ORF length, the stop codon set, and the sequence length.

But now let's be systematic: let's do an experiment where we vary the GC% from low to high. To reduce some sampling noise, for each GC%, I take the average over 10 synthetic genomes. It'd be nice to plot the result but we're not doing matplotlib yet. We'll just output a table. 

In [5]:
L          = 100000
ntrials    = 10
min_orflen = 200
print('{:>8s} {:>8s}'.format('GC', 'coverage'))
for gc in [ 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80 ]:
    avg_coverage = 0
    for i in range(ntrials):
        seq            = make_random_dna(rng, L, gc)
        coverage       = long_orf_coverage(seq, [ 'TAG', 'TAA', 'TGA' ], min_orflen)
        avg_coverage  += coverage
    avg_coverage /= ntrials
    print('{:8.2f} {:8.2f}'.format(gc, avg_coverage))
        

      GC coverage
    0.20     0.00
    0.30     0.00
    0.40     0.00
    0.50     0.00
    0.60     0.06
    0.70     0.59
    0.80     2.72


That shows the answer to the first part. GC% composition has a (perhaps surprisingly) strong effect on expected ORF lengths, because stop codons are AT-rich. $\geq$200aa is a reasonable threshold for a significant long ORF in low-GC or even 50% uniform composition DNA, so much so that you can just look for long ORFs to detect probable protein-coding regions. But in high-GC genomes, you expect lots of long ORFs just by chance, and many long ORFs will be noise, not real coding regions.

The SEA-PHAGE genomes happen to have very high-GC genomes, so their long ORF coverage is being jacked up by false positive ORFs. They probably have something like 0.7-0.8 coverage by real coding regions of $\geq 200$aa, with this noise added on. Hence when we look at Wiggins' stripplot, the yellow dots (around 50% GC) are around 0.5-1.0 coverage where we expect; the green dots (around 60% GC) have tacked on an extra 0.1-0.6 coverage from noise; and the purple dots (around 70% GC) have tacked on an extra 1-2.5x coverage from noise.

So if we're going to be using simple ORF length to detect probable coding regions, we're best off doing it in a low-GC, high-AT genome.


## 3. Download Moriarty's challenge of 10 phage genome sequences.

(I have these in my directory already - so did the TFs when they graded your psets. So we all skipped this step.)


## 4. Write a function to read a FASTA file and return the DNA sequence

Here's a quick and dirty one. It only reads one sequence per FASTA file, and it's not doing any error checking for format problems, but it'll work for my purposes.

In [6]:
def read_fasta_genome(seqfile):
    with open(seqfile) as f:
        line = f.readline()               # skip the first line that starts with > and has the name, description
        seq = ''
        for line in f:
            seq += line.strip().upper()   # .strip() removes whitespace; .upper() converts to upper case
    return seq


## 5. Take up Moriarty's challenge: screen the 10 phage for TAG- and TGA-recodings

We'll read each of Moriarty's 10 genomes one at a time. For each one, we calculate the fractional long ORF coverage statistic using three different genetic codes: standard code, TAG-recoded (only TAA|TGA stops), and TGA-recoded (only TAA|TAG stops). We output a table with those coverages, and also with the TAG/std and TGA/std ratios. If our hypothesis is right, we should be able to detect the recoded phage from them having outlying high coverage ratios.


In [7]:
phage_list = [ 'arugula', 'basil',  'chickpea', 'gooseberry', 'huckleberry', 
               'juniper', 'lentil', 'quince',   'sage',       'tangerine' ]
min_orflen = 200
print('{:12s} {:4s} {:4s} {:4s} {:7s} {:7s}'.format('phage', 'std', 'TAG', 'TGA', 'TAG/std', 'TGA/std'))
for phage in phage_list:
    seq          = read_fasta_genome(phage+'.fa')
    coverage_std = long_orf_coverage(seq, [ 'TAG', 'TAA', 'TGA' ], min_orflen)
    coverage_TAG = long_orf_coverage(seq, [        'TAA', 'TGA' ], min_orflen)
    coverage_TGA = long_orf_coverage(seq, [ 'TAG', 'TAA',       ], min_orflen)
    print('{:12s} {:4.2f} {:4.2f} {:4.2f} {:7.1f} {:7.1f}'.format(phage, coverage_std, coverage_TAG, coverage_TGA, 
                                                     coverage_TAG/ coverage_std, coverage_TGA / coverage_std))
    

phage        std  TAG  TGA  TAG/std TGA/std
arugula      0.37 0.40 0.74     1.1     2.0
basil        0.78 0.85 0.86     1.1     1.1
chickpea     0.84 0.88 0.92     1.0     1.1
gooseberry   0.29 0.78 0.39     2.7     1.3
huckleberry  0.80 0.86 0.84     1.1     1.1
juniper      0.29 0.75 0.35     2.6     1.2
lentil       0.37 0.38 0.74     1.0     2.0
quince       0.79 0.87 0.88     1.1     1.1
sage         0.84 0.93 0.92     1.1     1.1
tangerine    0.70 0.74 0.76     1.1     1.1


The recoded phage are easy to pick out here. The two TGA-recoded phage are arugula and lentil, with outlying TGA/std coverage ratios of 2.0. The two TAG-recoded phage are gooseberry and juniper, with outlying TAG/std coverage ratios of 2.7 and 2.6.

This doesn't necessarily mean that that's what's going on with Lestrade's phage, but it does mean it's a pretty plausible hypothesis.


## _Further reading_

Should you be inclined...

"Lestrade's" phage are 673 crAss phage genomes from the supplementary material of Yutin et al. (2001), ["_Analysis of metagenome-assembled viral genomes from the human gut reveals diverse putative CrAss-like phages with unique genomic features_"](https://doi.org/10.1038/s41467-021-21350-w). "crAss" stands for cross-assembly: this clade of abundant phage was first recognized when their genomes were assembled by combining ("cross-assembling")  unassigned contigs from many different bacterial genome assemblies. I used them instead of SEA-PHAGE for this pset for exactly the reasons you just saw: the simple long-ORF-coverage trick for screening for TAG/TGA-recoded phage doesn't work well in the SEA-PHAGE, which are generally GC-rich genomes. Lestrade's low long-ORF-coverage phage are indeed examples of TGA- and TAG-recoding. These alternative genetic codes are common in crAss phage.

The idea of using ORF coverage to detect phage genomes with recoded stop codons was introduced (I believe) by Ivanova et al. (2014) ["_Stop codon reassignments in the wild_"](https://doi.org/10.1126/science.1250691), and used in several discovery papers since, including a particularly nice one by Borges et al. (2022), ["_Widespread stop-codon recoding in bacteriophages may regulate translation of lytic genes_"](https://doi.org/10.1038/s41564-022-01128-6). These papers all use proper computational genefinding programs to infer their protein-coding ORFs, rather than simply taking any long ORF. For the pset, I wanted to make the idea work as simply as possible, just in basic Python, without needing some external program. I had to fiddle to find a really AT-rich phage dataset to make this pset work just by counting all long ORFs. We'll write our own probabilistic genefinder later in the course!





